<a href="https://colab.research.google.com/github/vivi-alencar/bachelor_thesis/blob/main/CLIP_UMAP_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pycocotools.coco import COCO

In [ ]:
from PIL import Image
import os
import albumentations as A
import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
import timm
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer

In [ ]:
import open_clip

In [ ]:
import json

In [ ]:
from collections import Counter

In [ ]:
import umap

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
from adjustText import adjust_text

In [ ]:
import joblib

In [ ]:
image_folder = "path to image folder" # Image folder

In [ ]:
image_files = os.listdir(image_folder)

In [ ]:
coco = COCO('path to instances_val2017.json')

In [ ]:
# Create a dictionary of to map category IDs to names
category_id_to_name = {category['id']: category['name'] for category in coco.loadCats(coco.getCatIds())}
class_queries = [f"a photo of a {name}" for name in category_id_to_name.values()]

In [ ]:
# Load the pretrained model, tokenizer, and preprocessing function
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
model.eval()  # Set to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Get all image IDs from COCO
image_ids = coco.getImgIds()
image_embeddings = []

# Iterate over each image
for image_id in tqdm(image_ids):
    # Get image file name from COCO
    image_info = coco.loadImgs(image_id)[0]
    image_path = f"{image_folder}/{image_info['file_name']}"

    # Open and preprocess the image
    image = Image.open(image_path).convert("RGB")
    image_input = preprocess(image).unsqueeze(0).to(device)

    # Generate the image embedding
    with torch.no_grad():
        image_embedding = model.encode_image(image_input)
    image_embeddings.append(image_embedding.cpu())  # Store on CPU to save GPU memory

# Stack all embeddings into a single tensor for later use
image_embeddings_tensor = torch.cat(image_embeddings)

In [ ]:
# Initialize lists to store image IDs and corresponding categories
image_ids_list = []
image_classes_list = []

# Populate lists with image IDs and the categories for each image
for image_id in image_ids:
    # Get all categories (annotations) for the image
    annotations = coco.loadAnns(coco.getAnnIds(imgIds=image_id))
    classes = {category_id_to_name[ann['category_id']] for ann in annotations}
    image_ids_list.append(image_id)
    image_classes_list.append(",".join(sorted(classes)))

# Create the DataFrame
coco_df = pd.DataFrame({
    'image_id': image_ids_list,
    'classes': image_classes_list
})

In [ ]:
# Create text embeddings
text_embeddings = []
for query in class_queries:
    # Tokenize and encode text
    text_tokens = open_clip.tokenize([query]).to(device)
    with torch.no_grad():
        text_embedding = model.encode_text(text_tokens)
    text_embeddings.append(text_embedding.cpu())

# Stack all text embeddings into a single tensor
text_embeddings_tensor = torch.cat(text_embeddings)

In [ ]:
print("Image embeddings shape:", image_embeddings_tensor.shape)  # Expected: [num_images, 512]
print("Text embeddings shape:", text_embeddings_tensor.shape)  # Expected: [num_categories, 512]

In [ ]:
# Normalize the image embeddings
image_embeddings_normalized = F.normalize(image_embeddings_tensor, p=2, dim=1)

# Normalize the text embeddings
text_embeddings_normalized = F.normalize(text_embeddings_tensor, p=2, dim=1)

In [ ]:
# Convert normalized embeddings to Numpy arrays
image_embeddings_np = image_embeddings_normalized.cpu().numpy()
text_embeddings_np = text_embeddings_normalized.cpu().numpy()

## Use UMAP

In [ ]:
# Initialize UMAP with parameters that often work well for image embeddings
umap_reducer = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, random_state=42, metric='cosine')

In [ ]:
umap_image_embeddings = umap_reducer.fit_transform(image_embeddings_np)
print("UMAP transformation completed.")

In [ ]:
## No colors
plt.figure(figsize=(8, 8))
plt.scatter(umap_image_embeddings[:, 0], umap_image_embeddings[:, 1], s=5, alpha=0.7)
plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.title("2D UMAP Projection of Image Embeddings")

plt.show()

In [ ]:
# Define classes and their corresponding colors
classes_to_highlight = {
    'skis': 'blue',
    'elephant': 'green',
    'airplane': 'purple',
    'sports ball': 'orange',
    'giraffe': 'red',
    'surfboard': 'pink',
    'train': 'cyan',
    'boat': 'navy',
    #'elephant': 'brown'
}

In [ ]:
# Plot all images in a light color
plt.figure(figsize=(8, 8))
plt.scatter(umap_image_embeddings[:, 0], umap_image_embeddings[:, 1], s=5, alpha=0.3, color='gray', label='Other Images')

# Plot each selected class with its unique color
for class_name, color in classes_to_highlight.items():
    class_df = coco_df[coco_df['classes'].str.contains(class_name)]
    class_indices = class_df.index.tolist()
    class_umap_embeddings = umap_image_embeddings[class_indices]

    plt.scatter(class_umap_embeddings[:, 0], class_umap_embeddings[:, 1], s=10, alpha=0.7, color=color, label=f'{class_name.capitalize()} Images')

plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.title("UMAP - Image Embeddings (ViT-B/32)")
plt.legend()

plt.show()

In [ ]:
## Filter images by category
giraffe_df = coco_df[coco_df['classes'].str.contains('giraffe')]
giraffe_indices = giraffe_df.index.tolist()  # Get indices of images with "giraffe"

In [ ]:
## Separate embeddings
giraffe_umap_image_embedding = umap_image_embeddings[giraffe_indices]

In [ ]:
## Visualize with specific color
plt.figure(figsize=(10, 7))

# Plot all images in a light color
plt.scatter(umap_image_embeddings[:, 0], umap_image_embeddings[:, 1], s=5, alpha=0.3, color='gray', label='Other Images')

# Plot only "giraffe" images in a distinct color
plt.scatter(giraffe_umap_image_embedding[:, 0], giraffe_umap_image_embedding[:, 1], s=10, alpha=0.7, color='red', label='Giraffe Images')

plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.title("2D UMAP Projection of Image Embeddings (Giraffe Highlighted)")
plt.legend()
plt.show()

## UMAP with PCA

In [ ]:
# Apply PCA to reduce dimensionality, then use UMAP
pca = PCA(n_components=50)  # You can try 50, 100, or another value less than 256
reduced_embeddings = pca.fit_transform(image_embeddings_np)
umap_reducer = umap.UMAP(n_components=2, n_neighbors=10, min_dist=0.4, random_state=42, metric="cosine")
umap_image_embeddings = umap_reducer.fit_transform(reduced_embeddings)

In [ ]:
## Visualize with specific color
plt.figure(figsize=(10, 7))

# Plot all images in a light color
plt.scatter(umap_image_embeddings[:, 0], umap_image_embeddings[:, 1], s=5, alpha=0.3, color='gray', label='Other Images')

# Plot only "giraffe" images in a distinct color
plt.scatter(giraffe_umap_image_embedding[:, 0], giraffe_umap_image_embedding[:, 1], s=10, alpha=0.7, color='red', label='Giraffe Images')

plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.title("2D UMAP Projection of Image Embeddings (Giraffe Highlighted)")
plt.legend()
plt.show()

## Text embeddings

In [ ]:
# Initialize UMAP with parameters suited for smaller datasets
umap_reducer = umap.UMAP(n_components=2, n_neighbors=5, min_dist=0.3, random_state=42, metric="cosine")
umap_text_embeddings = umap_reducer.fit_transform(text_embeddings_np)

In [ ]:
# Plot UMAP embeddings with labels
plt.figure(figsize=(12, 10))
plt.scatter(umap_text_embeddings[:, 0], umap_text_embeddings[:, 1], s=50, alpha=0.7)

# Create text annotations for each point and store them in a list
texts = []
for i, category_name in enumerate(category_id_to_name.values()):
    texts.append(
        plt.text(umap_text_embeddings[i, 0], umap_text_embeddings[i, 1], category_name, fontsize=10, ha='center')
    )

# Adjust text to reduce overlap
adjust_text(texts, only_move={'points': 'y', 'text': 'xy'}, arrowprops=dict(arrowstyle="->", color='grey', lw=0.5))

plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.title("2D UMAP Projection of COCO Category Text Embeddings")


plt.show()

### Attempt to color groups

In [ ]:
# Category to group mapping
category_to_group = {
    "airplane": "vehicle",
    "apple": "food",
    "backpack": "accessory",
    "banana": "food",
    "baseball glove": "sport",
    "baseball bat": "sport",
    "bear": "animal",
    "bed": "comfort",
    "bench": "comfort",
    "bicycle": "vehicle",
    "bird": "animal",
    "boat": "vehicle",
    #"book": "XXX",
    "bottle": "furniture",
    "bowl": "furniture",
    "broccoli": "food",
    "bus": "vehicle",
    "cake": "food",
    "car": "vehicle",
    "carrot": "food",
    "cat": "animal",
    "cell phone": "appliance",
    "chair": "comfort",
    #"clock": "XXX",
    "couch": "comfort",
    "cow": "animal",
    "cup": "furniture",
    "dining table": "furniture",
    "dog": "animal",
    "donut": "food",
    "elephant": "animal",
    "fire hydrant": "street",
    "fork": "cutlery",
    "frisbee": "sport",
    "giraffe": "animal",
    "hair drier": "appliance",
    "handbag": "accessory",
    "horse": "animal",
    "hot dog": "food",
    "keyboard": "appliance",
    "kite": "sport",
    "knife": "cutlery",
    "laptop": "appliance",
    "microwave": "appliance",
    "motorcycle": "vehicle",
    "mouse": "appliance",
    "orange": "food",
    "oven": "appliance",
    "parking meter": "street",
    #"person": "person",
    "pizza": "food",
    "potted plant": "furniture",
    "refrigerator": "appliance",
    "remote": "appliance",
    "sandwich": "food",
    "scissors": "cutlery",
    "sheep": "animal",
    "sink": "furniture",
    "skateboard": "sport",
    "skis": "sport",
    "snowboard": "sport",
    "spoon": "cutlery",
    "sports ball": "sport",
    "stop sign": "street",
    "suitcase": "accessory",
    "surfboard": "sport",
    #"teddy bear": "XXX",
    "tennis racket": "sport",
    #"tie": "XXX",
    "toaster": "appliance",
    "toilet": "furniture",
    #"toothbrush": "XXX",
    "traffic light": "street",
    "train": "vehicle",
    "truck": "vehicle",
    "tv": "appliance",
    #"umbrella": "XXX",
    "vase": "furniture",
    "wine glass": "furniture",
    "zebra": "animal",
}

In [ ]:
# Group to color mapping
group_to_color = {
    'vehicle': 'blue',
    'food': 'orange',
    'accessory': 'green',
    'sport': 'purple',
    'animal': 'red',
    'furniture': 'pink',
    'street': 'cyan',
    'cutlery': 'olive',
    'comfort': 'brown',
    'appliance': 'navy',
    'undefined': 'gray',  # Default color for undefined categories
}

In [ ]:
# Map category names to their corresponding colors
category_colors = []
for category in category_id_to_name.values():
    # Assign group or default to 'undefined'
    group = category_to_group.get(category, 'undefined')
    # Get the corresponding color
    category_colors.append(group_to_color[group])

In [ ]:
# Plot UMAP embeddings with labels and colors
plt.figure(figsize=(12, 10))
scatter = plt.scatter(
    umap_text_embeddings[:, 0],
    umap_text_embeddings[:, 1],
    s=50,
    c=category_colors,
    alpha=0.7
)

# Create text annotations for each point and store them in a list
texts = []
for i, category_name in enumerate(category_id_to_name.values()):
    texts.append(
        plt.text(umap_text_embeddings[i, 0], umap_text_embeddings[i, 1], category_name, fontsize=10, ha='center')
    )

# Adjust text to reduce overlap
adjust_text(texts, only_move={'points': 'y', 'text': 'xy'}, arrowprops=dict(arrowstyle="->", color='grey', lw=0.5))

plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.title("2D UMAP - COCO Category Text Embeddings (ViT-B/32)")

# Create a legend
handles = [plt.Line2D([0], [0], marker='o', color=color, markersize=10, linestyle='', label=group)
           for group, color in group_to_color.items()]
plt.legend(
    handles=handles,
    title="Groups",
    loc='best',
    prop={'size': 9},  # Adjust font size of the legend
    markerscale=0.7  # Adjust size of the legend markers
)


plt.show()

## Try UMAP - image and text embeddings together

In [ ]:
## We need to calculate image and text visualizations in one pass
combined_embeddings = np.vstack([image_embeddings_np, text_embeddings_np])

In [ ]:
## Run UMAP on combined embeddings
umap_reducer = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, metric="cosine")
combined_umap_embeddings = umap_reducer.fit_transform(combined_embeddings)

In [ ]:
# Separate the transformed embeddings
num_images = image_embeddings_np.shape[0]
umap_image_embeddings = combined_umap_embeddings[:num_images]
umap_text_embeddings = combined_umap_embeddings[num_images:]

In [ ]:
giraffe_index = list(category_id_to_name.values()).index("giraffe")
umap_giraffe_images = umap_image_embeddings[giraffe_indices]

In [ ]:
plt.figure(figsize=(10, 7))

# Plot all other image embeddings in a base color
plt.scatter(umap_image_embeddings[:, 0], umap_image_embeddings[:, 1], s=5, alpha=0.7, color='blue', label="Images")

# Plot all other text embeddings in a base marker/label style
plt.scatter(umap_text_embeddings[:, 0], umap_text_embeddings[:, 1], s=50, alpha=0.9, color='red', label="Categories")

# Set labels and title
plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.title("2D UMAP of Image and Text Embeddings (ViT-B/32)")
plt.legend()

plt.show()

## Try PCA

### Text only

In [ ]:
text_embeddings_np.shape

In [ ]:
# To decide the number of components, do variance calculations
# Perform PCA with enough components to capture most of the variance
text_pca = PCA(n_components=50)  # Adjust this to a high enough number
text_pca.fit(text_embeddings_np)

#### Explained variance

In [ ]:
# Plot individual explained variance for each component
text_explained_variance = text_pca.explained_variance_ratio_
print("Variance explained by each PCA component:", text_explained_variance)

plt.figure(figsize=(7, 4))
plt.bar(range(1, len(text_explained_variance) + 1), text_explained_variance, alpha=0.7)
plt.xlabel("Principal Component")
plt.ylabel("Explained Variance Ratio")
plt.title("Explained Variance - Text (ViT-B/32)")
plt.grid()


plt.show()

#### Cumulative variance

In [ ]:
# Calculate cumulative explained variance
text_cumulative_variance = np.cumsum(text_pca.explained_variance_ratio_)
print("Cumulative variance explained:", text_cumulative_variance)

# Plot cumulative explained variance
plt.figure(figsize=(7, 4))
plt.plot(range(1, len(text_cumulative_variance) + 1), text_cumulative_variance, marker='o', linestyle='--')
plt.xlabel("Number of Components")
plt.ylabel("Cumulative Explained Variance")
plt.title("Cumulative Explained Variance - Text (ViT-B/32)")
plt.grid()


plt.show()

In [ ]:
# Plot both cumulative and individual explained variance
plt.figure(figsize=(7, 4))
plt.plot(range(1, len(text_cumulative_variance) + 1), text_cumulative_variance, marker='o', linestyle='--', label='Cumulative Explained Variance')
plt.bar(range(1, len(text_explained_variance) + 1), text_explained_variance, alpha=0.6, label='Individual Explained Variance')

plt.xlabel("Number of Components")
plt.ylabel("Variance")
plt.title("Explained Variance - Text (ViT-B/32)")
plt.legend()
plt.grid()


plt.show()

In [ ]:
# Perform PCA transformation
text_embeddings_pca = text_pca.fit_transform(text_embeddings_np)

# Create the PCA scatter plot
plt.figure(figsize=(12, 10))
plt.scatter(text_embeddings_pca[:, 0], text_embeddings_pca[:, 1], s=50, alpha=0.7)

# Annotate points with category names using adjust_text
texts = []
for i, category_name in enumerate(category_id_to_name.values()):
    texts.append(
        plt.text(text_embeddings_pca[i, 0], text_embeddings_pca[i, 1], category_name, fontsize=10, ha='center')
    )

# Adjust text to reduce overlap
adjust_text(texts, only_move={'points': 'y', 'text': 'xy'}, arrowprops=dict(arrowstyle="->", color='grey', lw=0.5))

# Add labels and title
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("2D PCA Projection of COCO Category Text Embeddings")


# Show the plot
plt.show()

### Attempt to color

In [ ]:
# Create the PCA scatter plot with category colors
plt.figure(figsize=(12, 10))
plt.scatter(
    text_embeddings_pca[:, 0],
    text_embeddings_pca[:, 1],
    s=50,
    c=category_colors,  # Apply colors
    alpha=0.7
)

# Annotate points with category names using adjust_text
texts = []
for i, category_name in enumerate(category_id_to_name.values()):
    texts.append(
        plt.text(text_embeddings_pca[i, 0], text_embeddings_pca[i, 1], category_name, fontsize=10, ha='center')
    )

# Adjust text to reduce overlap
adjust_text(texts, only_move={'points': 'y', 'text': 'xy'}, arrowprops=dict(arrowstyle="->", color='grey', lw=0.5))

# Add labels and title
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("2D PCA - COCO Category Text Embeddings (ViT-B/32)")

# Create a legend
handles = [plt.Line2D([0], [0], marker='o', color=color, markersize=10, linestyle='', label=group)
           for group, color in group_to_color.items()]
plt.legend(
    handles=handles,
    title="Groups",
    loc='best',
    prop={'size': 9},  # Adjust font size of the legend
    markerscale=0.7  # Adjust size of the legend markers
)


# Show the plot
plt.show()

### Image only

In [ ]:
# Perform PCA with enough components to capture most of the variance
image_pca = PCA(n_components=50)  # Adjust this to a high enough number
image_pca.fit(image_embeddings_np)

In [ ]:
# Plot individual explained variance for each component
image_explained_variance = image_pca.explained_variance_ratio_
print("Variance explained by each PCA component:", image_explained_variance)

plt.figure(figsize=(7, 4))
plt.bar(range(1, len(image_explained_variance) + 1), image_explained_variance, alpha=0.7)
plt.xlabel("Principal Component")
plt.ylabel("Explained Variance Ratio")
plt.title("Explained Variance - Image (ViT-B/32)")
plt.grid()


plt.show()

In [ ]:
# Calculate cumulative explained variance
image_cumulative_variance = np.cumsum(image_pca.explained_variance_ratio_)
print("Cumulative variance explained:", image_cumulative_variance)

# Plot cumulative explained variance
plt.figure(figsize=(7, 4))
plt.plot(range(1, len(image_cumulative_variance) + 1), image_cumulative_variance, marker='o', linestyle='--')
plt.xlabel("Number of Components")
plt.ylabel("Cumulative Explained Variance")
plt.title("Cumulative Explained Variance - Image (ViT-B/32)")
plt.grid()


plt.show()

In [ ]:
# Plot both cumulative and individual explained variance
plt.figure(figsize=(7, 4))
plt.plot(range(1, len(image_cumulative_variance) + 1), image_cumulative_variance, marker='o', linestyle='--', label='Cumulative Explained Variance')
plt.bar(range(1, len(image_explained_variance) + 1), image_explained_variance, alpha=0.6, label='Individual Explained Variance')

plt.xlabel("Number of Components")
plt.ylabel("Variance")
plt.title("Explained Variance - Image (ViT-B/32)")
plt.legend()
plt.grid()


plt.show()

In [ ]:
# Plot PCA without colors
image_embeddings_pca = image_pca.fit_transform(image_embeddings_np)

plt.figure(figsize=(10, 7))
plt.scatter(image_embeddings_pca[:, 0], image_embeddings_pca[:, 1], s=5, alpha=0.7)

plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("2D PCA of COCO Image Embeddings")
plt.show()

In [ ]:
# Plot all images in a light color
plt.figure(figsize=(10, 7))
plt.scatter(image_embeddings_pca[:, 0], image_embeddings_pca[:, 1], s=5, alpha=0.3, color='gray', label='Other Images')

# Plot each selected class with its unique color
for class_name, color in classes_to_highlight.items():
    # Find indices for the class
    class_df = coco_df[coco_df['classes'].str.contains(class_name)]
    class_indices = class_df.index.tolist()

    # Get the PCA embeddings for the class
    class_pca_embeddings = image_embeddings_pca[class_indices]

    # Plot the points for the class
    plt.scatter(class_pca_embeddings[:, 0], class_pca_embeddings[:, 1],
                s=10, alpha=0.7, color=color, label=f'{class_name.capitalize()} Images')

# Add labels and title
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("2D PCA of COCO Image Embeddings (ViT-B/32)")
plt.legend()

plt.show()

## Try t-SNE with PCA

In [ ]:
# Initialize t-SNE and reduce PCA-transformed data to 2D
tsne = TSNE(n_components=2, random_state=42)
image_embeddings_tsnePCA = tsne.fit_transform(image_embeddings_pca)

In [ ]:
# Plot t-SNE results
plt.figure(figsize=(10, 7))
plt.scatter(image_embeddings_tsnePCA[:, 0], image_embeddings_tsnePCA[:, 1], s=5, alpha=0.7)
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.title("2D t-SNE of Image Embeddings")
plt.show()

In [ ]:
# Plot t-SNE results
plt.figure(figsize=(10, 7))
plt.scatter(image_embeddings_tsnePCA[:, 0], image_embeddings_tsnePCA[:, 1], s=5, alpha=0.3, color='gray', label='Other Images')

# Plot each selected class with its unique color
for class_name, color in classes_to_highlight.items():
    class_df = coco_df[coco_df['classes'].str.contains(class_name)]
    class_indices = class_df.index.tolist()
    class_tsnePCA_embeddings = image_embeddings_tsnePCA[class_indices]

    plt.scatter(class_tsnePCA_embeddings[:, 0], class_tsnePCA_embeddings[:, 1], s=10, alpha=0.7, color=color, label=f'{class_name.capitalize()} Images')

plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.title("2D t-SNE Projection of PCA-transformed Image Embeddings")
plt.legend()


plt.show()

# PCA GAP

In [ ]:
# Apply PCA to the combined embeddings
pca = PCA(n_components=2)
combined_pca_embeddings = pca.fit_transform(combined_embeddings)

# Separate the transformed embeddings
num_images = image_embeddings_np.shape[0]
pca_image_embeddings = combined_pca_embeddings[:num_images]
pca_text_embeddings = combined_pca_embeddings[num_images:]

# Plot the PCA-transformed embeddings
plt.figure(figsize=(10, 7))

# Plot the image embeddings in blue
plt.scatter(pca_image_embeddings[:, 0], pca_image_embeddings[:, 1], s=5, alpha=0.7, color='blue', label="Images")

# Plot the text embeddings in red
plt.scatter(pca_text_embeddings[:, 0], pca_text_embeddings[:, 1], s=50, alpha=0.9, color='red', label="Categories")


# Set labels and title
plt.xlabel("PCA Dimension 1")
plt.ylabel("PCA Dimension 2")
plt.title(f"2D PCA of Image and Text Embeddings (ViT-B/32)")
plt.legend()


plt.show()